# <center> <span style="color:red">Music recognition </span> </center>

The purpose of this notebook is to take in music spectrograms and to recognize musical genres

## Import 

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


## Import Model Linear and MLP 

In [2]:
import sys  
sys.path.insert(0, '../../python')

from linearModel import linearModel
from mlp import MLPModel

## Create Model 

In [3]:
lm = linearModel()
mlp = MLPModel()

## Algorithme 

In [4]:
def import_images_and_assign_labels(folder, label, X, Y):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8, 8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)

In [5]:
def import_dataset():
    dataset_folder = "G:/Programmes/Python/projetAnnuel/data/"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Electro"), [1,0,0], X_train, y_train
    )
    import_images_and_assign_labels(
        os.path.join(train_folder, "Metal"), [0,1,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Rap"), [0,0,1], X_train, y_train
    )
    
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "Electro"), [1,0,0], X_test, y_test
    )
    import_images_and_assign_labels(
        os.path.join(test_folder, "Metal"), [0,1,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "Rap"), [0,0,1], X_test, y_test
    )

    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))

In [6]:
def display_value(number,array):
    print("Sur le dataset de Train")
    cpt = 0
    for i in array:
        print(cpt," ",np.argmax(i))
        cpt+=1
        if(cpt==number):
            cpt=0

In [17]:
def run_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat1 = mlp.create_mlp_model([192,8,8,3])
    model1 = resultat1[0]
        
    flattened_dataset_inputs = []
    for p in X_train:
        for i in p:
            flattened_dataset_inputs.append(i)
            
    flattened_dataset_outputs = []
    for p in y_train:
        for i in p:
            flattened_dataset_outputs.append(i)
   
    mlp.train_classification_stochastic_gradient_backpropagation(model1,
                                                                flattened_dataset_inputs,
                                                                flattened_dataset_outputs,
                                                                alpha=1e-2,
                                                                iterations_count=2000000)

    predicted_outputs = [mlp.predict_mlp_model_classification(model1, p) for p in X_test]
    
#     item1 = 100
#     item2 = 100
#     for i in range(0,len(predicted_outputs)):
#         if i > 9:
#             if predicted_outputs[i] > 0:
#                 item1 -= 10
#         else:
#             if predicted_outputs[i] < 0:
#                 item2 -= 10
        
    #print(predicted_outputs)
    display_value(10,predicted_outputs)
    
    mlp.free_MLP(resultat1)
    
    #print(f"il y a {item2}% de chanson identifié en électro")
    #print(f"il y a {item1}% de chanson identifié en métal")

In [18]:
run_mlp()

Sur le dataset de Train
0   0
1   0
2   0
3   0
4   2
5   2
6   2
7   0
8   0
9   0
0   1
1   1
2   1
3   1
4   1
5   1
6   2
7   2
8   1
9   0
0   2
1   2
2   2
3   0
4   0
5   2
6   2
7   0
8   0
9   0
